In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time
from ema_workbench import save_results, load_results
from ema_workbench import (Model, RealParameter, perform_experiments, ScalarOutcome)

from ema_workbench import (
    Model,
    MultiprocessingEvaluator,
    ScalarOutcome,
    IntegerParameter,
    optimize,
    Scenario,
)
from ema_workbench.em_framework.optimization import EpsilonProgress
from ema_workbench.util import ema_logging

from problem_formulation import get_model_for_problem_formulation
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(2)

In [3]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [4]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [5]:
levers = copy.deepcopy(dike_model.levers)
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('Expected Annual Damage', variable_name=('A.1_Expected Annual Damage', 'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage', 'A.5_Expected Annual Damage'), function=<function sum_over at 0x000001871943EA20>)
ScalarOutcome('Dike Investment Costs', variable_name=('A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs'), function=<function sum_over at 0x000001871943EA20>)
ScalarOutcome('RfR Investment Costs', variable_name=('RfR Total Costs',), function=<function sum_over at 0x000001871943EA20>)
ScalarOutcome('Evacuation Costs', variable_name=('Expected Evacuation Costs',), function=<function sum_over at 0x000001871943EA20>)
ScalarOutcome('Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths', 'A.3_Expected Number of Deaths', 'A.4_Expected Number of Deaths', 'A.5_Expected Number of Deaths'), function

In [6]:
policy = [Policy({'0_RfR 0': 1.0, '0_RfR 1': 0.0, '0_RfR 2': 1.0, '1_RfR 0': 0.0, '1_RfR 1': 0.0, '1_RfR 2': 0.0, '2_RfR 0': 1.0, '2_RfR 1': 1.0, '2_RfR 2': 1.0, '3_RfR 0': 1.0, '3_RfR 1': 1.0, '3_RfR 2': 1.0, '4_RfR 0': 0.0, '4_RfR 1': 0.0, '4_RfR 2': 1.0, 'EWS_DaysToThreat': 2.0, 'A.1_DikeIncrease 0': 0.0, 'A.1_DikeIncrease 1': 0.0, 'A.1_DikeIncrease 2': 0.0, 'A.2_DikeIncrease 0': 3.0, 'A.2_DikeIncrease 1': 0.0, 'A.2_DikeIncrease 2': 0.0, 'A.3_DikeIncrease 0': 0.0, 'A.3_DikeIncrease 1': 0.0, 'A.3_DikeIncrease 2': 0.0, 'A.4_DikeIncrease 0': 1.0, 'A.4_DikeIncrease 1': 0.0, 'A.4_DikeIncrease 2': 1.0, 'A.5_DikeIncrease 0': 2.0, 'A.5_DikeIncrease 1': 0.0, 'A.5_DikeIncrease 2': 0.0})]

In [7]:
# policy = [Policy({'0_RfR 0': 1.0, '0_RfR 1': 0.0, '0_RfR 2': 1.0, '1_RfR 0': 0.0, '1_RfR 1': 0.0, '1_RfR 2': 0.0, '2_RfR 0': 1.0, '2_RfR 1': 1.0, '2_RfR 2': 1.0, '3_RfR 0': 1.0, '3_RfR 1': 1.0, '3_RfR 2': 1.0, '4_RfR 0': 0.0, '4_RfR 1': 0.0, '4_RfR 2': 1.0, 'EWS_DaysToThreat': 2.0, 'A.1_DikeIncrease 0': 0.0, 'A.1_DikeIncrease 1': 0.0, 'A.1_DikeIncrease 2': 0.0, 'A.2_DikeIncrease 0': 3.0, 'A.2_DikeIncrease 1': 0.0, 'A.2_DikeIncrease 2': 0.0, 'A.3_DikeIncrease 0': 0.0, 'A.3_DikeIncrease 1': 0.0, 'A.3_DikeIncrease 2': 0.0, 'A.4_DikeIncrease 0': 1.0, 'A.4_DikeIncrease 1': 0.0, 'A.4_DikeIncrease 2': 1.0, 'A.5_DikeIncrease 0': 2.0, 'A.5_DikeIncrease 1': 0.0, 'A.5_DikeIncrease 2': 0.0}),
#  Policy({'0_RfR 0': 0.0, '0_RfR 1': 1.0, '0_RfR 2': 0.0, '1_RfR 0': 0.0, '1_RfR 1': 0.0, '1_RfR 2': 0.0, '2_RfR 0': 1.0, '2_RfR 1': 1.0, '2_RfR 2': 1.0, '3_RfR 0': 1.0, '3_RfR 1': 1.0, '3_RfR 2': 1.0, '4_RfR 0': 0.0, '4_RfR 1': 0.0, '4_RfR 2': 0.0, 'EWS_DaysToThreat': 3.0, 'A.1_DikeIncrease 0': 0.0, 'A.1_DikeIncrease 1': 0.0, 'A.1_DikeIncrease 2': 0.0, 'A.2_DikeIncrease 0': 2.0, 'A.2_DikeIncrease 1': 0.0, 'A.2_DikeIncrease 2': 0.0, 'A.3_DikeIncrease 0': 0.0, 'A.3_DikeIncrease 1': 0.0, 'A.3_DikeIncrease 2': 0.0, 'A.4_DikeIncrease 0': 3.0, 'A.4_DikeIncrease 1': 0.0, 'A.4_DikeIncrease 2': 0.0, 'A.5_DikeIncrease 0': 1.0, 'A.5_DikeIncrease 1': 0.0, 'A.5_DikeIncrease 2': 0.0, 'Expected Annual Damage': 0.0, 'Dike Investment Costs': 76309037.45427957, 'RfR Investment Costs': 540300000.0, 'Evacuation Costs': 0.0, 'Expected Number of Deaths': 0.0})]

In [ ]:
from ema_workbench import MultiprocessingEvaluator, ema_logging
from ema_workbench import Samplers
from ema_workbench.em_framework import get_SALib_problem

ema_logging.log_to_stderr(ema_logging.INFO)
    
with MultiprocessingEvaluator(dike_model, n_processes=-2) as evaluator:
    experiments, results = evaluator.perform_experiments(128, policies=policy,
                                                         uncertainty_sampling=Samplers.SOBOL)

[MainProcess/INFO] pool started with 14 workers
[MainProcess/INFO] performing 5120 scenarios * 1 policies * 1 model(s) = 5120 experiments
  0%|                                                 | 0/5120 [00:00<?, ?it/s]

In [ ]:
problem = get_SALib_problem(dike_model.uncertainties)
problem

In [ ]:
from SALib.analyze.sobol import analyze

In [ ]:
sobol_results = {}
for policy in experiments.policy.unique():
    logical = experiments.policy == policy
    y = results['reliability'][logical]
    indices = analyze(problem, y)
    sobol_results[policy] = indices

In [ ]:
fig, axes = plt.subplots(1,3)
fig.set_size_inches(8, 4)
indices.plot(ax=axes)

## Feature score

In [ ]:
from ema_workbench.analysis import feature_scoring

In [ ]:
cleaned_experiments = experiments.drop(columns=[l.name for l in dike_model.levers])

scores = feature_scoring.get_feature_scores_all(cleaned_experiments,results)

In [ ]:
import seaborn as sns
cleaned_experiments = experiments.drop(columns=[l.name for l in dike_model.levers])

for policy in experiments.policy.unique():
    logical = experiments.policy == policy
    subset_results = {k:v[logical] for k,v in results.items()}
    scores = feature_scoring.get_feature_scores_all(cleaned_experiments[logical],
                                                   subset_results)
    sns.heatmap(scores, annot=True, cmap='viridis')
    plt.show()